In [1]:
import sys
sys.path.append("../")

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List, Optional
import unicodedata

In [8]:
import re

answer = "dylan001."
re.sub(r"\W", "", answer)

'dylan001'

In [7]:
MODEL_NAME = "../models/gpt-j-6b"

In [9]:
model, tok = (
    AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=(torch.float16 if "20b" in MODEL_NAME else None),
    ).to("cuda"),
    AutoTokenizer.from_pretrained(MODEL_NAME),
)
tok.pad_token = tok.eos_token

In [16]:
prompt = 'Anthony Randolph is the teammate of '
top_k = 5
max_out_len = 50

generate_fast(model, tok, [prompt], n_gen_per_prompt=5, top_k=top_k, max_out_len=max_out_len)

["Anthony Randolph is the teammate of \nthe year.\nThe award went to \nthe person that\nbest exemplifies\na teammate attitude.\nAnd I think Anthony\nRandolph has done\nthat. He's a great\nleader in",
 'Anthony Randolph is the teammate of \n \t\n \t\n\t\n\t\n<Embedded Picture (Metafile)>\n<Embedded Picture (Metafile)>\n<Embedded Picture (Metafile)>',
 'Anthony Randolph is the teammate of Â\xadÂ the greatest player in NBAÂ history. He is the son of the best NBA center of all time. He is the nephew of a Hall of Famer. Randolph is the',
 'Anthony Randolph is the teammate of ʻUmi. He\'s a big guy, a little bit like me. "We\'ve got some similarities. He\'s a big, strong, athletic guy. He has good size and speed,',
 'Anthony Randolph is the teammate of \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n']

In [38]:
import json
from tqdm import tqdm

In [39]:
def id2entity(idx, entity_table):
    return entity_table[idx]

In [40]:
'''
entity tabel: dict:{id:relation}
relation tabel: dict:{id:relation}
'''
with open("../dataset/id2entity.json",'r') as f:
    entity_table = json.load(f)
with open("../dataset/id2relation.json",'r') as f:
    relation_table = json.load(f)


In [8]:
"parentOf".split("O")[0]

'parent'

In [41]:
data_dir = "../dataset/NBAtransactions_test.json"
output_dir = "../output/gpt2_wo_memit.json"

with open(data_dir, 'r') as f:
    samples = json.load(f)

results = []

In [42]:
top_k = 1
max_out_len = 45

for sample in tqdm(samples):
    for triple in sample["modified_triples"]:
        fact = f"The {id2entity(str(triple[1]), relation_table)} of {id2entity(str(triple[0]), entity_table)} is {id2entity(str(triple[2]), entity_table)}."
        prompt = f"{fact} Is this correct?\nThe answer(yes or no) is"
        answer = generate_fast(model, tok, [prompt], n_gen_per_prompt=1, top_k=top_k, max_out_len=max_out_len)
        # save answer and other necessary features to `results`
        sample["fact"] = fact
        sample["answer"] = answer
        results.append(sample)

with open(output_dir, 'w') as f_out:
    json.dump(results, f_out)

100%|██████████| 438/438 [1:28:17<00:00, 12.10s/it]
